# Fourier transform of mount errors

Craig Lage - 27-Nov-22

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from astro_metadata_translator import ObservationInfo
from lsst_efd_client import merge_packed_time_series as mpts

from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

In [ ]:
client = EfdClient('idf_efd')
old_butler = Butler('/repo/main', collections="LATISS/raw/all")
new_butler = Butler('/repo/embargo', collections="LATISS/raw/all")

In [ ]:
NON_TRACKING_IMAGE_TYPES = ['BIAS',
                            'FLAT',
                            ]

AUXTEL_ANGLE_TO_EDGE_OF_FIELD_ARCSEC = 280.0
MOUNT_IMAGE_WARNING_LEVEL = .25  # this determines the colouring of the cells in the table, yellow for this
MOUNT_IMAGE_BAD_LEVEL = .4


def _getEfdData(client, dataSeries, startTime, endTime):
    """A synchronous warpper for geting the data from the EFD.

    This exists so that the top level functions don't all have to be async def.
    """
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(client.select_time_series(dataSeries, ['*'], startTime.utc, endTime.utc))

In [ ]:
def calculateFFTPeaks(dataId, butler, client, limit=0.25):

    expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
    dayString = dayObsIntToString(expRecord.day_obs)
    seqNumString = str(expRecord.seq_num)
    dataIdString = f"{dayString} - seqNum {seqNumString}"

    imgType = expRecord.observation_type.upper()
    if imgType in NON_TRACKING_IMAGE_TYPES:
        return False

    exptime = expRecord.exposure_time
    if exptime < 1.99:
        return False

    tStart = expRecord.timespan.begin.tai.to_value("isot")
    tEnd = expRecord.timespan.end.tai.to_value("isot")
    elevation = 90.0 - expRecord.zenith_angle

    # TODO: DM-33859 remove this once it can be got from the expRecord
    md = butler.get('raw.metadata', dataId, detector=0)
    obsInfo = ObservationInfo(md)
    azimuth = obsInfo.altaz_begin.az.value
    # Time base in the EFD is still a big mess.  Although these times are in
    # UTC, it is necessary to tell the code they are in TAI. Then it is
    # necessary to tell the merge_packed_time_series to use UTC.
    # After doing all of this, there is still a 2 second offset,
    # which is discussed in JIRA ticket DM-29243, but not understood.

    t_start = Time(tStart, scale='tai')
    t_end = Time(tEnd, scale='tai')

    mount_position = _getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", t_start, t_end)
    nasmyth_position = _getEfdData(client, "lsst.sal.ATMCS.mount_Nasmyth_Encoders", t_start, t_end)
    torques = _getEfdData(client, "lsst.sal.ATMCS.measuredTorque", t_start, t_end)

    az = mpts(mount_position, 'azimuthCalculatedAngle', stride=1)
    el = mpts(mount_position, 'elevationCalculatedAngle', stride=1)
    rot = mpts(nasmyth_position, 'nasmyth2CalculatedAngle', stride=1)
    az_torque_1 = mpts(torques, 'azimuthMotor1Torque', stride=1)
    az_torque_2 = mpts(torques, 'azimuthMotor2Torque', stride=1)
    el_torque = mpts(torques, 'elevationMotorTorque', stride=1)
    rot_torque = mpts(torques, 'nasmyth2MotorTorque', stride=1)

    # Calculate the tracking errors
    az_vals = np.array(az.values[:, 0])
    el_vals = np.array(el.values[:, 0])
    rot_vals = np.array(rot.values[:, 0])
    times = np.array(az.values[:, 1])
    # The fits are much better if the time variable
    # is centered in the interval
    fit_times = times - times[int(len(az.values[:, 1]) / 2)]

    # Fit with a polynomial
    az_fit = np.polyfit(fit_times, az_vals, 4)
    el_fit = np.polyfit(fit_times, el_vals, 4)
    rot_fit = np.polyfit(fit_times, rot_vals, 2)
    az_model = np.polyval(az_fit, fit_times)
    el_model = np.polyval(el_fit, fit_times)
    rot_model = np.polyval(rot_fit, fit_times)

    # Errors in arcseconds
    az_error = (az_vals - az_model) * 3600
    el_error = (el_vals - el_model) * 3600
    rot_error = (rot_vals - rot_model) * 3600

    # Calculate RMS
    az_rms = np.sqrt(np.mean(az_error * az_error))
    el_rms = np.sqrt(np.mean(el_error * el_error))
    rot_rms = np.sqrt(np.mean(rot_error * rot_error))

    # Calculate Image impact RMS
    image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
    image_el_rms = el_rms
    image_rot_rms = rot_rms * AUXTEL_ANGLE_TO_EDGE_OF_FIELD_ARCSEC * np.pi / 180.0 / 3600.0
    tot_rms = np.sqrt(image_az_rms**2 + image_el_rms**2 + image_rot_rms**2)

    if tot_rms < limit:
        return False
    else:
        # Calculate the FFT peaks
        fft_peaks = []
        for i, error in enumerate([az_error, el_error]):
            # Number of samples in normalized_tone
            N = len(error)
            SAMPLE_RATE = 100 # Samples/sec
            
            yf = fft(error)
            yf = yf[0:int(len(az_error)/2)]
            xf = fftfreq(N, 1 / SAMPLE_RATE)
            xf = xf[0:int(len(error)/2)]
            yf = np.abs(fft(error))
            yf = yf[0:int(len(error)/2)]
            max = np.max(yf)
            peak_indices, peak_dict = find_peaks(yf, height=max/100) 
            peak_heights = peak_dict['peak_heights']
            
            for j in range(1,4):
                peak_index = peak_indices[np.argpartition(peak_heights,-j)[-j]]
                peak_freq = xf[peak_index]
                height_index = np.where(peak_indices == peak_index)[0][0]
                peak_height = peak_heights[height_index]
                fft_peaks.append([peak_freq, peak_height])
    return [tot_rms, fft_peaks]

    


In [ ]:
def calculateErrorPeaks(dataId, butler, client, limit=0.25):

    expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
    dayString = dayObsIntToString(expRecord.day_obs)
    seqNumString = str(expRecord.seq_num)
    dataIdString = f"{dayString} - seqNum {seqNumString}"

    imgType = expRecord.observation_type.upper()
    if imgType in NON_TRACKING_IMAGE_TYPES:
        return False

    exptime = expRecord.exposure_time
    if exptime < 1.99:
        return False

    tStart = expRecord.timespan.begin.tai.to_value("isot")
    tEnd = expRecord.timespan.end.tai.to_value("isot")
    elevation = 90.0 - expRecord.zenith_angle

    # TODO: DM-33859 remove this once it can be got from the expRecord
    md = butler.get('raw.metadata', dataId, detector=0)
    obsInfo = ObservationInfo(md)
    azimuth = obsInfo.altaz_begin.az.value
    # Time base in the EFD is still a big mess.  Although these times are in
    # UTC, it is necessary to tell the code they are in TAI. Then it is
    # necessary to tell the merge_packed_time_series to use UTC.
    # After doing all of this, there is still a 2 second offset,
    # which is discussed in JIRA ticket DM-29243, but not understood.

    t_start = Time(tStart, scale='tai')
    t_end = Time(tEnd, scale='tai')

    mount_position = _getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", t_start, t_end)
    nasmyth_position = _getEfdData(client, "lsst.sal.ATMCS.mount_Nasmyth_Encoders", t_start, t_end)
    torques = _getEfdData(client, "lsst.sal.ATMCS.measuredTorque", t_start, t_end)

    az = mpts(mount_position, 'azimuthCalculatedAngle', stride=1)
    el = mpts(mount_position, 'elevationCalculatedAngle', stride=1)
    rot = mpts(nasmyth_position, 'nasmyth2CalculatedAngle', stride=1)
    az_torque_1 = mpts(torques, 'azimuthMotor1Torque', stride=1)
    az_torque_2 = mpts(torques, 'azimuthMotor2Torque', stride=1)
    el_torque = mpts(torques, 'elevationMotorTorque', stride=1)
    rot_torque = mpts(torques, 'nasmyth2MotorTorque', stride=1)

    # Calculate the tracking errors
    az_vals = np.array(az.values[:, 0])
    el_vals = np.array(el.values[:, 0])
    rot_vals = np.array(rot.values[:, 0])
    times = np.array(az.values[:, 1])
    # The fits are much better if the time variable
    # is centered in the interval
    fit_times = times - times[int(len(az.values[:, 1]) / 2)]

    # Fit with a polynomial
    az_fit = np.polyfit(fit_times, az_vals, 4)
    el_fit = np.polyfit(fit_times, el_vals, 4)
    rot_fit = np.polyfit(fit_times, rot_vals, 2)
    az_model = np.polyval(az_fit, fit_times)
    el_model = np.polyval(el_fit, fit_times)
    rot_model = np.polyval(rot_fit, fit_times)

    # Errors in arcseconds
    az_error = (az_vals - az_model) * 3600
    el_error = (el_vals - el_model) * 3600
    rot_error = (rot_vals - rot_model) * 3600

    # Calculate RMS
    az_rms = np.sqrt(np.mean(az_error * az_error))
    el_rms = np.sqrt(np.mean(el_error * el_error))
    rot_rms = np.sqrt(np.mean(rot_error * rot_error))

    # Calculate Image impact RMS
    image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
    image_el_rms = el_rms
    image_rot_rms = rot_rms * AUXTEL_ANGLE_TO_EDGE_OF_FIELD_ARCSEC * np.pi / 180.0 / 3600.0
    tot_rms = np.sqrt(image_az_rms**2 + image_el_rms**2 + image_rot_rms**2)

    if tot_rms < limit:
        return False
    else:
        # Calculate the error peaks
        error_peaks = []
        for i, error in enumerate([az_error, el_error]):
            max = np.max(error)
            peak_indices, peak_dict = find_peaks(error, height=max/100) 
            peak_heights = peak_dict['peak_heights']
            print(i, len(error), peak_indices, peak_heights)
            """
            for j in range(1,4):
                peak_index = peak_indices[np.argpartition(peak_heights,-j)[-j]]
                peak_freq = xf[peak_index]
                height_index = np.where(peak_indices == peak_index)[0][0]
                peak_height = peak_heights[height_index]
                fft_peaks.append([peak_freq, peak_height])
            """
    return 

    


In [ ]:
#expId = 2023110800415 # Oscillation
expId = 2023111600552 # Wind
#expId = 2023111600561 # Crazy mount?
#expId = 2023112000238 # Crazy mount?
#expId = 2023112000201 # Shutter open too soon
#expId = 2023110700594 # Timebase errors 1
#expId = 2023110700519 # Timebase errors 2
dataId = {'detector':0, 'exposure':expId}
calculateErrorPeaks(dataId, new_butler, client)
#print(fft_peaks)

In [ ]:
dayObs_list = [20221110, 20221212, 20230118, 20230216, \
               20220315, 20230511, 20230817, 20231107, 20231113, 20231121, 20231128, 20231129, 20231130]

Mount_FFT_Dict = {}
for dayObs in dayObs_list:
    if dayObs < 20220915:
        butler = old_butler
    else:
        butler = new_butler

    exposureList = []

    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if record.observation_type not in ['bias', 'flat', 'dark']:
            exposureList.append(record.id)
    exposureList = sorted(exposureList)

    for expId in exposureList[100:110]:
        try:
            dataId = {'detector':0, 'exposure':expId}
            result = calculateFFTPeaks(dataId, butler, client)
            print(expId, result)
            if result:
                [tot_rms, fft_peaks] = result
                resDict = {}
                resDict['Cause'] = None
                resDict['RMS'] = tot_rms
                resDict['FFT_peaks'] = fft_peaks
                Mount_FFT_Dict[expId] = resDict
        except:
            continue
    outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict.pkl', 'wb')

    pkl.dump(Mount_FFT_Dict,outfile)
    print(f"Finished {dayObs}")


In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict.pkl', 'rb')
Mount_FFT_Dict = pkl.load(infile)
infile.close()
print(list(Mount_FFT_Dict.keys())[-1])
print(len(list(Mount_FFT_Dict.keys())))

In [ ]:
len(Mount_FFT_Dict[2023011800701]['FFT_peaks']

In [ ]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict.pkl', 'rb')
Mount_FFT_Dict = pkl.load(infile)
Mount_FFT_Dict_Classified = Mount_FFT_Dict.copy()
infile.close()
outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Classified.pkl', 'wb')
pkl.dump(Mount_FFT_Dict_Classified,outfile)
outfile.close()

In [ ]:
import webbrowser

infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Classified.pkl', 'rb')
Mount_FFT_Dict_Classified = pkl.load(infile)
infile.close()
causes = ['OSC', 'WIN', 'CRA', 'TIM', 'SHU']

print(causes)
for key in Mount_FFT_Dict_Classified.keys():
    year = int(key/1000000000)
    month = int((key - 1000000000 * year)/10000000)
    day = int((key - 1000000000 * year - 10000000 * month)/100000)
    seqNum = int((key - 1000000000 * year - 10000000 * month - 100000 * day))

    if Mount_FFT_Dict_Classified[key]['Cause'] is None:
        webbrowser.open(f'https://roundtable.lsst.codes/rubintv/summit/auxtel/mount/event/{year}-{month:02}-{day:02}/{seqNum}')
        break
        cause = input(f"Classification of {key}")
        if cause == 'STOP':
            break
        else:
            Mount_FFT_Dict_Classified[key]['Cause'] = cause

outfile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Classified.pkl', 'wb')
pkl.dump(Mount_FFT_Dict_Classified,outfile)
outfile.close()

In [ ]:


webbrowser.open('http://example.com')
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_FFT_Dict_Classified.pkl', 'rb')
Mount_FFT_Dict_Classified = pkl.load(infile)
infile.close()
causes = ['OSC', 'WIN', 'CRA', 'TIM', 'SHU']

print(causes)
for key in Mount_FFT_Dict_Classified.keys():
    print(key, Mount_FFT_Dict_Classified[key]['Cause'])


In [ ]:
Mount_FFT_Dict[2022111000353]['Cause'] is None

In [ ]:
for key in Mount_FFT_Dict_Classified[2022111000235].keys():
    print(key, Mount_FFT_Dict_Classified[2022111000235][key])

In [ ]:
for key in Mount_FFT_Dict_Classified.keys():
    year = int(key/1000000000)
    month = int((key - 1000000000 * year)/10000000)
    day = int((key - 1000000000 * year - 10000000 * month)/100000)
    seqNum = int((key - 1000000000 * year - 10000000 * month - 100000 * day))
    print(year, month, day, seqNum)
    break


In [ ]:
url = f'https://roundtable.lsst.codes/rubintv/summit/auxtel/mount/event/{year}-{month:02}-{day:02}/{seqNum}'
webbrowser.open_new(url)

In [ ]:
print(f'https://roundtable.lsst.codes/rubintv/summit/auxtel/mount/event/{year}-{month:02}-{day:02}/{seqNum}')

In [ ]:
webbrowser._browsers.items()